In [38]:
from fastai.vision import *
import numpy as np
import torch.utils.data as data_utils
import torch
from sklearn import preprocessing
import cv2
from torchvision.utils import make_grid

In [39]:
class NumbersDataset():
    def __init__(self, inputs, labels):
        self.X = inputs
        self.y = labels
        self.c = 15


    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        img_train = cv2.imread(self.X[idx])
        img_mask = cv2.imread(self.y[idx])
        img_train = cv2.resize(img_train, (224,224), interpolation = cv2.INTER_LANCZOS4)
        img_mask = cv2.resize(img_mask, (224,224), interpolation = cv2.INTER_LANCZOS4)
        return img_train, img_mask

In [40]:
class CustomTensorDataset(Dataset):

    def __init__(self, tensors, transform=None):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform
        self.c = 15

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]

        c = 15

        return x, y, c

    def __len__(self):
        return self.tensors[0].size(0)

In [4]:
X_train = np.load("../data/generator/generated_data.npy")
Y_train = np.load("../data/generator/generated_data_labels.npy")

In [5]:
le = preprocessing.LabelEncoder()
le.fit(Y_train)
Y_train = le.transform(Y_train)
#le.inverse_transform(Y_train)

In [6]:
X_train = torch.from_numpy(X_train)
Y_train = torch.Tensor(Y_train)

In [7]:
X_train.size(), Y_train.size()

(torch.Size([80820, 100, 100]), torch.Size([80820]))

In [8]:
#train_ds = data_utils.TensorDataset(X_train, Y_train)
train_ds = CustomTensorDataset(tensors = (X_train, Y_train))
#train_ds = NumbersDataset(X_train, Y_train)

In [9]:
train_ds

In [10]:
test_set = []
test_set_labels = []

#Read in sample.
for filename in os.listdir("../KNN/data/symbols_test/"):
    img = cv2.imread("../KNN/data/symbols_test/" + filename,0)
    name = re.findall('([a-zA-Z_ ]*)\d*.*', filename)[0]
    #Preprocess image by removing blank columns and rows from edges
    img = img[np.argwhere(np.amin(img,axis=1) < 120)[0][0]:np.argwhere(np.amin(img,axis=1) < 120)[-1][0],:]
    img = img[:,np.argwhere(np.amin(img,axis=0) < 120)[0][0]:np.argwhere(np.amin(img,axis=0) < 120)[-1][0]]
    img = cv2.resize(img, [100,100])
    test_set.append(img)
    test_set_labels.append(name)

In [11]:
Y_test = le.transform(np.array(test_set_labels))

In [12]:
X_test = torch.from_numpy(np.array(test_set))
Y_test = torch.Tensor(Y_test)

In [13]:
#test_ds = data_utils.TensorDataset(X_test)
test_ds = CustomTensorDataset(tensors = (X_test, Y_test))
#test_ds = NumbersDataset(X_test, Y_test)

In [36]:
dataloader_train = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=0, pin_memory=False)
dataloader_test = DataLoader(test_ds, batch_size=32, shuffle=True, num_workers=0, pin_memory=False)

TypeError: Dataset() takes no arguments

In [32]:
data = ImageDataBunch.create(dataloader_train, dataloader_test)
#data = ImageDataBunch(X_train, Y_train)
#data = ImageDataBunch(train_ds, test_ds)

In [33]:
data

ImageDataBunch;

Train: <__main__.CustomTensorDataset object at 0x0000017796D6BC70>;

Valid: <__main__.CustomTensorDataset object at 0x0000017796D628E0>;

Test: None

In [34]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy)

In [37]:
learn.fit_one_cycle(3)

RuntimeError: DataLoader worker (pid(s) 5656, 4860, 14772, 6728, 7888, 10860, 15076, 5624, 12692, 6152, 2664, 9212) exited unexpectedly

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)